In [4]:
import pandas as pd
from rpy2.robjects import r, pandas2ri
import rpy2.robjects as ro
from rpy2.robjects.conversion import localconverter

# Load and clean data
df = pd.read_csv("/Users/sambrown04/Documents/SURF/whillans-surf/notebooks/SURF/neural_nets/averages_events_2011-13.csv", parse_dates=["start_time"])
df = df.iloc[1:]  # Drop header 

# Keep only selected columns
columns_needed = [
    'mins_since', 'tide_change', 'tide_height', 'h_event',
    'form_factor', 'slip_severity', 'total_delta', 'pre_slip_area', 'mins_until'
]
df = df[columns_needed].dropna()

# Pass to R
with localconverter(ro.default_converter + pandas2ri.converter):
    r_df = ro.conversion.py2rpy(df)

ro.globalenv["data"] = r_df

# Run linear regression in R with specified predictors
r('model <- lm(mins_until ~ mins_since + tide_change + tide_height + h_event + form_factor + slip_severity + total_delta + pre_slip_area, data=data)')
r('summary_output <- summary(model)')
r('print(summary_output)')


Error importing in API mode: ImportError("dlopen(/opt/anaconda3/lib/python3.12/site-packages/_rinterface_cffi_api.abi3.so, 0x0002): Library not loaded: /Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib\n  Referenced from: <733B2185-C608-342F-8FC5-CF619524A0A2> /opt/anaconda3/lib/python3.12/site-packages/_rinterface_cffi_api.abi3.so\n  Reason: tried: '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file)")
Trying to import in ABI mode.
/opt/anaconda3/lib/python3.12/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "PWD" redefined by R and overriding existing variable. Current: "/Users/sambrown04/Documents/SURF/whillans-surf", R: "/Users/sambrown04/Documents


Call:
lm(formula = mins_until ~ mins_since + tide_change + tide_height + 
    h_event + form_factor + slip_severity + total_delta + pre_slip_area, 
    data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-738.03 -232.82  -70.52  303.12  949.91 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    8.142e+02  6.322e+01  12.879  < 2e-16 ***
mins_since    -1.854e-01  8.593e-02  -2.157  0.03113 *  
tide_change   -1.711e+02  6.599e+01  -2.593  0.00961 ** 
tide_height    1.370e+00  3.492e-01   3.922 9.13e-05 ***
h_event        3.522e+01  3.070e+01   1.147  0.25141    
form_factor    9.876e+00  1.643e+00   6.010 2.29e-09 ***
slip_severity  2.935e+08  4.238e+07   6.927 6.18e-12 ***
total_delta   -1.248e+02  3.276e+02  -0.381  0.70321    
pre_slip_area  1.098e-01  1.002e-01

/opt/anaconda3/lib/python3.12/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "R_SESSION_TMPDIR" redefined by R and overriding existing variable. Current: "/var/folders/zg/jhrvqkbd33x0c7lyzw9rdfsc0000gn/T//RtmpktZsES", R: "/var/folders/zg/jhrvqkbd33x0c7lyzw9rdfsc0000gn/T//RtmpCthNgB"
  warnings.warn(


   1.097  0.27293    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 310.9 on 1633 degrees of freedom
Multiple R-squared:  0.06269,	Adjusted R-squared:  0.0581 
F-statistic: 13.65 on 8 and 1633 DF,  p-value: < 2.2e-16



Interesting... Tide_h, form_factor, slip_sev all seem to be most significant. Slip severity seems to be a good predictor. I am unsure why. Overall, very bad model

In [6]:
r('model <- lm(mins_until ~ slip_severity, data=data)')
r('summary_output <- summary(model)')
r('print(summary_output)')



Call:
lm(formula = mins_until ~ slip_severity, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-559.45 -245.26  -89.02  305.61  915.58 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)   8.350e+02  2.411e+01  34.633  < 2e-16 ***
slip_severity 1.361e+08  2.662e+07   5.114 3.53e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 317.9 on 1640 degrees of freedom
Multiple R-squared:  0.01569,	Adjusted R-squared:  0.01509 
F-statistic: 26.15 on 1 and 1640 DF,  p-value: 3.531e-07

